In [1]:
import pandas as pd
from sqlalchemy import create_engine, text, inspect
from sqlalchemy import Column, Integer, String, Float, ForeignKey,Date
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Session
Base = declarative_base()

g:\programs\Anaconda\envs\PythonData\lib\site-packages\numpy\__init__.py:140: UserWarning: mkl-service package failed to import, therefore Intel(R) MKL initialization ensuring its correct out-of-the box operation under condition when Gnu OpenMP had already been loaded by Python process is not assured. Please install mkl-service package, see http://github.com/IntelPython/mkl-service
  from . import _distributor_init


ImportError: Unable to import required dependencies:
numpy: 

IMPORTANT: PLEASE READ THIS FOR ADVICE ON HOW TO SOLVE THIS ISSUE!

Importing the numpy C-extensions failed. This error can happen for
many reasons, often due to issues with your setup or how NumPy was
installed.

We have compiled some common reasons and troubleshooting tips at:

    https://numpy.org/devdocs/user/troubleshooting-importerror.html

Please note and check the following:

  * The Python version is: Python3.7 from "g:\programs\Anaconda\envs\PythonData\python.exe"
  * The NumPy version is: "1.18.5"

and make sure that they are the versions you expect.
Please carefully study the documentation linked above for further help.

Original error was: DLL load failed: The specified module could not be found.


In [ ]:
dept_path = r"data/departments.csv"
depemp_path = r"data/dept_emp.csv"
deptman_path = r"data/dept_manager.csv"
emp_path = r"data/employees.csv"
salary_path = r"data/salaries.csv"
title_path = r"data/titles.csv"

: 

In [ ]:
depts = pd.read_csv(dept_path)
# depts.head()

: 

In [ ]:
class Title(Base):
    __tablename__ = "titles"
    id =  Column(Integer, primary_key=True)
    title_id = Column(String(255))
    title = Column(String(255))

: 

In [ ]:
class Department(Base):
    __tablename__ = "departments"
    id =  Column(Integer, primary_key=True)
    dept_no = Column(String(255))
    dept_name = Column(String(255))

: 

In [ ]:
class Employee(Base):
    __tablename__ = "employees"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(String(255))
    emp_title_id = Column(Integer, ForeignKey(Title.title_id),primary_key = True) 
    birth_date = Column(Date)
    first_name = Column(String(255))
    last_name = Column(String(255))
    sex = Column(String(255))
    hire_date = Column(Date)

: 

In [ ]:
class Manager(Base):
    __tablename__ = "managers"
    id = Column(Integer, primary_key=True)
    dept_no = Column(String(255), ForeignKey(Department.dept_no), primary_key = True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)

: 

In [ ]:
class Salary(Base):
    __tablename__ = "salaries"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    salary = Column(Integer)

: 

In [ ]:
class Department_Employee(Base):
    __tablename__ = "department_employees"
    id =  Column(Integer, primary_key=True)
    emp_no = Column(Integer, ForeignKey(Employee.emp_no), primary_key = True)
    dept_no = Column(String(255), ForeignKey(Department.dept_no), primary_key = True)

: 

In [ ]:
engine = create_engine('sqlite:///data/johnson.sqlite')
Base.metadata.create_all(engine)

: 

In [ ]:
session = Session(bind=engine)

: 

In [ ]:
# dep0 = Department(dept_no = depts.iloc[[0]]["dept_no"][0], dept_name = depts.iloc[[0]]["dept_name"][0])
# print(type(dep0.dept_name))
# session.add(dep0)
# session.commit()

: 

In [ ]:
def depf(ind,df):
    return Department(dept_no = df.iloc[[ind]]["dept_no"][0], dept_name = df.iloc[[ind]]["dept_name"][0])

: 

In [ ]:
def titlef(ind,df):
    return Title(title_id = df.iloc[ind]["title_id"], title = df.iloc[ind]["title"])

: 

In [ ]:
def depempf(ind,df):
    return Department_Employee(dept_no = df.iloc[[ind]]["dept_no"][0], emp_no = df.iloc[[ind]]["emp_no"][0])

: 

In [ ]:
def depmanf(ind,df):
    return Manager(dept_no = df.iloc[[ind]]["dept_no"][0], emp_no = df.iloc[[ind]]["emp_no"][0])

: 

In [ ]:
def salf(ind,df):
    return Salary(emp_no = df.iloc[[ind]]["emp_no"][0], salary = df.iloc[[ind]]["salary"][0])

: 

In [ ]:
def empf(ind,df):
    return Employee(emp_no = df.iloc[[ind]]["emp_no"][0], 
                    emp_title = df.iloc[[ind]]["emp_title"][0],
                    birth_date = df.iloc[[ind]]["birth_date"][0],
                    first_name = df.iloc[[ind]]["first_name"][0], 
                    last_name = df.iloc[[ind]]["last_name"][0],
                    sex = df.iloc[[ind]]["sex"][0],
                    hire_date = df.iloc[[ind]]["hire_date"][0], )

: 

In [ ]:
from sqlalchemy import inspect
# inspector = inspect(engine)
# inspector.get_table_names()
# inspector.get_columns('departments')
deptso = session.query(Title)
# that_one = session.query(Department).filter(Department.dept_name == "Marketing").first()
# session.delete(that_one)
for dpt in deptso:
    print(dpt.title)
session.commit()

: 

In [ ]:
titles = session.query(Title)
titles_df=pd.read_csv(title_path)

: 

In [ ]:

# for i in range(4):
    # session.add(titlef(j,titles_df))
session.add(titlef(0,titles_df))
session.add(titlef(2,titles_df))
session.add(titlef(3,titles_df))
    
session.commit()


: 

In [ ]:

titles = session.query(Title)
for t in titles:
    print(t.title)

: 